## Imports

In [37]:
from datasets import Dataset
from keras.utils.np_utils import to_categorical

## Constantes

In [38]:
dir = "../../"
dir_data = "data/"
dir_rotulacao = "rotulacao/result/"

arquivo_result = "facebook_12.json" ## Verificar caso a caso

dir_dataset = "datasets/dataset_facebook/"## Verificar caso a caso
dataset_name = "dataset_facebook.json" ## Verificar caso a caso

## Lendo Resultados

In [39]:
resultado_doccano = Dataset.from_json(f"{dir}{dir_data}{dir_rotulacao}{arquivo_result}")

Using custom data configuration default-8c2f5157974dfc06
Found cached dataset json (/home/thiago/.cache/huggingface/datasets/json/default-8c2f5157974dfc06/0.0.0)


In [40]:
resultado_doccano

Dataset({
    features: ['id', 'text', 'index', 'label', 'Comments'],
    num_rows: 12
})

## Lendo Dataset

In [41]:
dataset = Dataset.from_json(f"{dir}{dir_data}{dir_dataset}{dataset_name}")

Using custom data configuration default-f68307eca29a7ccb


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /home/thiago/.cache/huggingface/datasets/json/default-f68307eca29a7ccb/0.0.0. Subsequent calls will reuse this data.


In [42]:
dataset

Dataset({
    features: ['created_time', 'text', 'permalink_url', 'id_post', 'dataset_origem', 'repetido', 'labels', 'labels_int', 'index'],
    num_rows: 34620
})

## Prepara Dados Resultados

In [43]:
resultado_doccano.features

{'id': Value(dtype='int64', id=None),
 'text': Value(dtype='string', id=None),
 'index': Value(dtype='int64', id=None),
 'label': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'Comments': Sequence(feature=Value(dtype='null', id=None), length=-1, id=None)}

### Arrumar Colunas de Label

In [44]:
resultado_doccano = resultado_doccano.add_column("labels_int", [None] * len(resultado_doccano))

In [45]:
def arruma_label(data):
    valor = 0
    if "Neutro" in data["label"]:
        valor = 1
    elif "Positivo" in data["label"]:
        valor = 2
    data["label"] = to_categorical(valor, num_classes=3)
    data["labels_int"] = valor
    return data
resultado_doccano = resultado_doccano.map(arruma_label)

Loading cached processed dataset at /home/thiago/.cache/huggingface/datasets/json/default-8c2f5157974dfc06/0.0.0/cache-14160a2952510aeb.arrow


In [46]:
resultado_doccano.features

{'id': Value(dtype='int64', id=None),
 'text': Value(dtype='string', id=None),
 'index': Value(dtype='int64', id=None),
 'label': Sequence(feature=Value(dtype='float32', id=None), length=-1, id=None),
 'Comments': Sequence(feature=Value(dtype='null', id=None), length=-1, id=None),
 'labels_int': Value(dtype='int64', id=None)}

## Prepara Dados Dataset

In [47]:
dataset.features

{'created_time': Value(dtype='string', id=None),
 'text': Value(dtype='string', id=None),
 'permalink_url': Value(dtype='string', id=None),
 'id_post': Value(dtype='string', id=None),
 'dataset_origem': Value(dtype='string', id=None),
 'repetido': Value(dtype='bool', id=None),
 'labels': Sequence(feature=Value(dtype='float64', id=None), length=-1, id=None),
 'labels_int': Value(dtype='float64', id=None),
 'index': Value(dtype='int64', id=None)}

### Adiciona a coluna de "label"

In [48]:
if "labels" not in dataset.features:
    dataset = dataset.add_column("labels", [None] * len(dataset))
    dataset = dataset.add_column("labels_int", [None] * len(dataset))

In [49]:
dataset.features

{'created_time': Value(dtype='string', id=None),
 'text': Value(dtype='string', id=None),
 'permalink_url': Value(dtype='string', id=None),
 'id_post': Value(dtype='string', id=None),
 'dataset_origem': Value(dtype='string', id=None),
 'repetido': Value(dtype='bool', id=None),
 'labels': Sequence(feature=Value(dtype='float64', id=None), length=-1, id=None),
 'labels_int': Value(dtype='float64', id=None),
 'index': Value(dtype='int64', id=None)}

## Merge - Dataset & Result Doccano

### Convertendo result Doccano para Dict

In [50]:
dict_resultado_doccano = dict({dado["index"] : [dado["label"], dado["labels_int"], dado["text"]] for dado in resultado_doccano})

### Merge

In [51]:
def merge(data):
    if data["index"] in dict_resultado_doccano.keys() and not data["repetido"]:
        data["labels"] = dict_resultado_doccano[data["index"]][0]
        data["labels_int"] = dict_resultado_doccano[data["index"]][1]
        # print(f"texto doccano: {dict_resultado_doccano[data['index']][2]}")
        # print(f"texto dataset: {data['text']}")
        # print("-----------------------------------")
    return data
dataset = dataset.map(merge)

  0%|          | 0/34620 [00:00<?, ?ex/s]

## Salvando Dataset

In [52]:
dataset.to_json(f"{dir}{dir_data}{dir_dataset}{dataset_name}")

Creating json from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

14128029

## Testes

In [53]:
from collections import Counter
print(Counter(dataset["labels_int"]))

Counter({None: 33417, 1.0: 593, 2.0: 579, 0.0: 31})


In [54]:
if "repetido" in dataset.features:
    dataset_repetido = dataset.filter(lambda data: data["repetido"])
    print(Counter(dataset_repetido["labels_int"]))

  0%|          | 0/35 [00:00<?, ?ba/s]

Counter({None: 180})


### Remover

In [55]:
texto_remover = ""

In [56]:
def remover_labels_by_texto(data, texto_remover: str, coluna: str):
    if texto_remover in data[coluna] and data["labels"]:
        data["labels"] = None
        data["labels_int"] = None
        print(f"Removeu labels do texto: {data[coluna]}")
    return data

dataset = dataset.map(lambda data: remover_labels_by_texto(data, texto_remover, "text"))


  0%|          | 0/34620 [00:00<?, ?ex/s]

Removeu labels do texto: Nesta meu figo ta morto
